In the second tutorial, we wanted to find a decomposition that would allow us to approximate a matrix using only some of its columns.  We will calculate this using a column-pivoted QR factorization.  

## What is a QR factorization?  
A QR factorization is a factorization of a matrix such that $A=QR$, where $Q$ is a matrix with orthonormal columns, and $R$ is an upper triangular matrix.  I will explain why this form is helpful for finding the interpolative decomposition later on.

Software that performs a QR factorization is readily available in scipy.  Take a look at the example below:  


In [ ]:
import numpy as np
import scipy.linalg as ln

np.random.seed(0)
A=np.random.random((4,4))-.5
print("Example Matrix A:")
print(A)

Q,R=ln.qr(A)

print("R:")
print(R)
print("Q:")
print(Q)
print("Checking Orthogonality:")
print("Q^TQ:")
print((Q.T@Q).round(4))

print("QR")
print(Q@R)

Look at $R$ above: an 'upper triangular matrix' has zeros as entries below the diagonal. 

It is important to realize that QR factorization is not unique.  There are multiple sets of $Q$ and $R$ matrices that would work.

## How do we find a QR factorization?  
Multiple methods of finding QR factorizations exist.  The one that will likely seem most familiar is the Gram-Schmidt process.  

### Gram-Schmidt

In linear algebra, we usually learn about the Gram-Schmidt method to orthonormalize vectors.  This method treats the columns of $A$ as a set of vectors to orthonormalize in order to get a QR factorization.  

In the first step of the Gram-Schmidt process, we select the first column and normalize it.  This becomes the first column in our $Q$, and the norm of the first column becomes the top left value in the $R$ that we are constructing.  

$ A= \begin{bmatrix} \frac{A_{:,1}}{||A_{:,1}||} & ... \end{bmatrix} \begin{bmatrix} ||A_{:,1}||&...\\ 0 &\end{bmatrix}$

We want the next column in $Q$ to be orthonormal to the first, so we take the next column in $A$, subtract off the part that is not orthogonal to the first column, and then normalize it.  

The second column in $Q$ becomes:  

$Q_{:,2}=\frac{A_{:,2}-(A_{:,2}*Q_{:,1})Q_{:,1}}{||A_{:,2}-(A_{:,2}*Q_{:,1})Q_{:,1}||}$

In order to maintain $A=QR$, the second column of $R$ becomes:  

$R= \begin{bmatrix} ||A_{:,1}||& A_{:,2} Q_{:,1} & ...\\ 0 & A_{:,2} Q_{:,2}& ... \\ 0 &0\end{bmatrix}$

This process is iterative. For each subsequent step, we start with the next column of $A$, and then we orthogonalize it to all of the existing columns of $Q$ by subtracting off the parts of the column that are not orthogonal to each of the columns.  We then write $R$ so that we can recover $A$.    


### Rank-Revealing QR (Bussinger and Golub 1965)  

The way to find the interpolative decomposition (the reason we went through all of the above) is by finding a rank-revealing QR factorization. The method that we use to find a rank-revealing QR factorization is called column-pivoted QR.  This is the same algorithm as the Gram-Schmidt method, except at each step we choose the largest remaining column, and "pivot" it (move it) to the "working vector" that we are orthogonalizing to the columns in $Q$.  We keep track of where the columns go by using a permutation matrix.  

Instead of $A=QR$, we get $A=QR\Pi^T$.  

This is also readily available in scipy. Note that $\Pi$ is not returned in matrix form, but is instead a list of indices to save space.  If $\Pi$ were [2 1 3 0], that means we put the third column in the first position, the second column in the second position, etc.

In the code below, we use $P$ instead of $\Pi$ for simplicity.

In [ ]:
np.random.seed(0)
A=np.random.random((4,4))-.5
print("Example Matrix A:")
print(A)

Q,R, P=ln.qr(A, pivoting=True)
print("P")
print(P)
np.argsort(P)
print("R:")
print(R)
print("Q:")
print(Q)
print("Checking Orthogonality:")
print("Q^TQ:")
print((Q.T@Q).round(4))

print("QRP^T")

print((Q@R)[:,np.argsort(P)])

### Why do QR factorizations give us an interpolative decomposition? 


Now that we have a rank-revealing QR factorization, we can use it to find an interpolative decomposition.  Say we want to find an approximation of $A$ using $k$ columns of $A$.  


We start with $A \Pi =QR$

We can partition this into submatrices, so that 

$\Pi = [\Pi_1 \ \Pi_2]$

$Q= [Q_1 \ Q_2]$

$R= \begin{bmatrix}
R_{11} & R_{12} \\
0 & R_{22}
\end{bmatrix} $

Where $R_{11}$ is $k\times k$.  

Therefore, we have:  

$A [\Pi_1 \ \Pi_2] = [Q_1 \ Q_2] \begin{bmatrix}
R_{11} & R_{12} \\
0 & R_{22}
\end{bmatrix}$


Our column-pivoted QR factorization tries to make sure that $R_{22}$ is as small as possible, and in practice usually gets close.  


We use the permutation $\Pi_1$ to select the columns of $A$.  

$A_{:,I}=A \Pi_1$

By writing out the partitions, we also see that:  

$A_{:,I}=Q_{1}R_{11}$.  

(though in practice we want to use $A \Pi_1$ to find $A_{:,I}$ since it is cheaper to compute.)

We then use $R$ along with the permutation to find our interpolation matrix:  

$T=\begin{bmatrix}I_k & R_{11}^{-1} R_{12}\end{bmatrix}\Pi^T$

Now, we can find our error.  

$||A-A_{:,I}T||= ||\begin{bmatrix} Q_1 & Q_2 \end{bmatrix}  \begin{bmatrix} R_{11} & R_{12} \\0 & R_{22}\end{bmatrix} \begin{bmatrix} \Pi_1 \\ \Pi_2\end{bmatrix} - Q_1 R_{11} \begin{bmatrix} I_k & R_{11}^{-1}R_{12}\end{bmatrix} \begin{bmatrix} \Pi_1 \\ \Pi_2\end{bmatrix} ||$

$||A-A_{:,I}T||=||\begin{bmatrix} 0 & Q_2 R_{22 } \Pi_2 \end{bmatrix}||= ||R_{22}||$.  


Therefore, we have an approximation of $A$, made of a subset of columns of $A$, and an interpolating matrix which can interpolate the unused columns.  We also have controllable error.  We will use this in the next tutorial to prune our networks!  